In [2]:
!pip install opencv-python

   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
    --------------------------------------- 0.8/39.5 MB 7.1 MB/s eta 0:00:06
   - -------------------------------------- 1.6/39.5 MB 3.7 MB/s eta 0:00:11
   -- ------------------------------------- 2.4/39.5 MB 3.8 MB/s eta 0:00:10
   -- ------------------------------------- 2.9/39.5 MB 3.4 MB/s eta 0:00:11
   --- ------------------------------------ 3.7/39.5 MB 3.5 MB/s eta 0:00:11
   ---- ----------------------------------- 4.7/39.5 MB 3.6 MB/s eta 0:00:10
   ----- ---------------------------------- 5.5/39.5 MB 3.6 MB/s eta 0:00:10
   ------ --------------------------------- 6.3/39.5 MB 3.7 MB/s eta 0:00:10
   ------- -------------------------------- 7.1/39.5 MB 3.7 MB/s eta 0:00:09
   ------- -------------------------------- 7.9/39.5 MB 3.7 MB/s eta 0:00:09
   -------- ------------------------------- 8.7/39.5 MB 3.7 MB/s eta 0:00:09
   --------- ------------------------------ 9.4/39.5 MB 3.8 MB/s eta 0:00:08
   ---

In [3]:
!pip install pillow

   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   ----------- ---------------------------- 0.8/2.6 MB 5.2 MB/s eta 0:00:01
   ----------------------- ---------------- 1.6/2.6 MB 4.1 MB/s eta 0:00:01
   ----------------------------------- ---- 2.4/2.6 MB 3.7 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 3.7 MB/s eta 0:00:00


In [11]:
import cv2
import tkinter as tk
from tkinter import filedialog, Label, Button
from PIL import Image, ImageTk, ImageEnhance, ImageOps

# Initialize webcam
cap = cv2.VideoCapture(0)
photos = []  # Store the three captured photos

# Apply a vintage filter
def apply_vintage_filter(img):
    """Apply a vintage effect to an image."""
    img = Image.fromarray(img)
    img = ImageOps.colorize(img.convert("L"), black="#3e2723", white="#ffccbc")  # Sepia tone
    img = ImageEnhance.Brightness(img).enhance(1.2)  # Increase brightness
    img = ImageEnhance.Contrast(img).enhance(1.3)  # Boost contrast
    return img

# Capture a photo
def capture_photo():
    """Capture a single photo and apply the vintage filter."""
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        filtered_img = apply_vintage_filter(frame)
        photos.append(filtered_img)
        
        if len(photos) <= 3:
            status_label.config(text=f"Photo {len(photos)} captured! Take {3 - len(photos)} more.")
        if len(photos) == 3:
            status_label.config(text="3 photos captured! Click 'Show Collage' to see them.")
    else:
        status_label.config(text="Failed to capture photo. Try again!")

# Show the collage
def show_collage():
    """Create and display a polaroid-style collage."""
    if len(photos) == 3:
        collage = Image.new("RGB", (1000, 400), "#fffde7")  # Cream background
        border = 10
        
        # Place each photo in the collage with a border
        for i, photo in enumerate(photos):
            photo_resized = photo.resize((300, 300))
            polaroid = Image.new("RGB", (320, 340), "#ffffff")  # White frame like a polaroid
            polaroid.paste(photo_resized, (border, border))
            
            # Add the polaroid to the collage
            collage.paste(polaroid, (i * 330 + 10, 20))
        
        # Save and show the collage
        collage.show()
        file_path = filedialog.asksaveasfilename(defaultextension=".png", 
                                                 filetypes=[("PNG files", ".png"), ("JPEG files", ".jpg")])
        if file_path:
            collage.save(file_path)
            status_label.config(text=f"Collage saved as: {file_path}")
    else:
        status_label.config(text="Capture 3 photos first!")

# Update the live video preview
def update_frame():
    """Show the live webcam feed."""
    ret, frame = cap.read()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        img = Image.fromarray(frame)
        img_tk = ImageTk.PhotoImage(image=img)
        video_label.img_tk = img_tk
        video_label.config(image=img_tk)
    video_label.after(10, update_frame)

# Create the GUI window
root = tk.Tk()
root.title("Vintage Polaroid PhotoBooth")
root.geometry("1000x600")
root.configure(bg="#ffebee")

# Add UI elements
Label(root, text="📸 Vintage Polaroid PhotoBooth 📸", font=("Comic Sans MS", 20, "bold"), bg="#f48fb1", fg="#fff").pack(pady=10)

video_label = Label(root, bg="#fff")
video_label.pack()

Button(root, text="✨ Capture Photo ✨", command=capture_photo, font=("Comic Sans MS", 14), bg="#f06292", fg="#fff").pack(pady=10)

Button(root, text="🖼 Show Collage 🖼", command=show_collage, font=("Comic Sans MS", 14), bg="#ba68c8", fg="#fff").pack(pady=10)

status_label = Label(root, text="Capture 3 photos to create a collage!", font=("Comic Sans MS", 12), bg="#ffebee", fg="#d81b60")
status_label.pack(pady=10)

# Start the live video
update_frame()
root.mainloop()

# Release the webcam
cap.release()
cv2.destroyAllWindows()